## Referans: https://github.com/HilalGozutok/RFM-ANALYSIS

In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Virgülden sonra gösterilecek olan sayı. (Opsiyonel)
# pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [181]:
# Recency = “Bir müşteri en son ne zaman alışveriş yapmış?” 
# Frequency = “Müşteri ne sıklıkla alışveriş yapıyor?” 
# Monetary = “Müşteri herbir alışverişinde toplam ne kadar para bırakmış?” 


InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.

StockCode: Ürün kodu. Her bir ürün için eşsiz numara.

Description: Ürün ismi

Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.

InvoiceDate: Fatura tarihi ve zamanı.

UnitPrice: Ürün fiyatı (Sterlin cinsinden)

CustomerID: Eşsiz müşteri numarası

Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [182]:
df_2010_2011 = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")
df = df_2010_2011.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


# Dataframe’i Anlamaya Yönelik Sorular

In [184]:
#essiz urun sayisi nedir?
df["Description"].nunique()

4223

In [185]:
#hangi urunden kacar tane var?
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     2369
REGENCY CAKESTAND 3 TIER               2200
JUMBO BAG RED RETROSPOT                2159
PARTY BUNTING                          1727
LUNCH BAG RED RETROSPOT                1638
                                       ... 
Missing                                   1
historic computer difference?....se       1
DUSTY PINK CHRISTMAS TREE 30CM            1
WRAP BLUE RUSSIAN FOLKART                 1
PINK BERTIE MOBILE PHONE CHARM            1
Name: Description, Length: 4223, dtype: int64

In [186]:
#en cok siparis edilen urunler hangisi?
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039
...,...
Damaged,-7540
Printing smudges/thrown away,-9058
check,-12030


In [187]:
#toplam kac fatura kesilmiştir?
df["Invoice"].nunique()


25900

In [188]:
# Feature Engineering : Veri setinde görüken ya da gözükmeyen yeni değişkenler 
# türetmektir. Bazen bu mevcut değişkenler üzerinden bazende yapısal
# olmayan farklı kaynaklardan olur.
#fatura basina toplam kac para kazanilmistir?

df["TotalPrice"] = df["Quantity"] * df["Price"]
df.groupby("Invoice").agg({"TotalPrice": "sum"})


,TotalPrice
Invoice,
536365,139
536366,22
536367,279
536368,70
536369,18
...,...
C581484,-168470
C581490,-33
C581499,-225


In [189]:
#en cok iade alan urun hangisidir?
# df[df["Invoice"].str[0] == "C"].sort_values("Quantity")
df[df["Invoice"].astype(str).str.startswith("C")].sort_values("Quantity").head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2,16446,United Kingdom,-168470
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1,12346,United Kingdom,-77184
4268,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0,15838,United Kingdom,-281
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2,15749,United Kingdom,-6539
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,2,15749,United Kingdom,-3700


In [190]:
# "Invoice" değişenine odaklanın ve C ile başlayanların iade ürünler olduğunu bilin
# ve onlardan kurtulmaya çalışın.
# Bu kod ile C olmayanları al ve dataframe tekrar atama yap dedik.

df = df[~df["Invoice"].astype(str).str.startswith("C")]
df


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2,12680,France,13
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4,12680,France,17
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4,12680,France,17
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,5,12680,France,15


# Veri Ön İşleme (Data Preprocessing)

Veri Ön İşleme (Data Preprocessing)

In [191]:
# Kaç eksik değer var?
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    134697
Country             0
TotalPrice          0
dtype: int64

In [192]:
# Modelleme yapmadığımız için eksik değerlerin bir önemi yok
df.dropna(inplace=True)

print(df.isnull().any())
print("Hiç eksik değer kaldı mı?")
print(df.isnull().values.any())

Invoice        False
StockCode      False
Description    False
Quantity       False
InvoiceDate    False
Price          False
Customer ID    False
Country        False
TotalPrice     False
dtype: bool
Hiç eksik değer kaldı mı?
False


Aykırı Değerler (Outliers)

ML işlemi yapmayacağımız için aykırı değerlerinde anlamı düşmüş oldu. Aykırı değerlere işlem yapmıyoruz çünkü RFM Skorlarını aykırı değerlere göre oluşturacağız.

In [193]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T


,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397925,13,180,1,1,1,1,2,6,12,24,36,120,80995
Price,397925,3,22,0,0,0,1,1,2,4,6,8,15,8143
Customer ID,397925,15294,1713,12346,12415,12627,12883,13969,15159,16795,17725,17912,18211,18287
TotalPrice,397925,22,309,0,1,1,2,5,12,20,35,68,202,168470


In [194]:
# Aykırı değerleri bulmaya yarayan kod
for feature in ["Quantity", "Price", "TotalPrice"]:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature, "yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")


Quantity yes
963
Price yes
661
TotalPrice yes
903


Recency Hesaplama

In [195]:
# Veri setindeki minimum değer.
df["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [196]:
# Veri setine max değeri.
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [197]:
# Veri setine göre bu max değeri bugünün tarihi olarak atayalım.
import datetime as dt
today_date = dt.datetime(2011, 12, 9)
today_date


datetime.datetime(2011, 12, 9, 0, 0)

In [198]:
df.groupby("Customer ID").agg({"InvoiceDate": "max"}).head()

,InvoiceDate
Customer ID,
12346,2011-01-18 10:01:00
12347,2011-12-07 15:52:00
12348,2011-09-25 13:13:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00


In [199]:
# Customer ID integer çevirdik.
df["Customer ID"] = df["Customer ID"].astype(int)

# Bugünden müşteri özelinde son satın alınma tarihini çıkarma işlemi yaptık.
# df.groupby("Customer ID").agg({"InvoiceDate": "max"}).apply(lambda x: today_date - x)
(today_date - df.groupby("Customer ID").agg({"InvoiceDate": "max"})).head()


,InvoiceDate
Customer ID,
12346,324 days 13:59:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [200]:
# temp_df = df.groupby("Customer ID").agg({"InvoiceDate": "max"}).apply(lambda x: today_date - x)
temp_df = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))
temp_df.rename(columns = {"InvoiceDate": "Recency"}, inplace = True)
temp_df.head()


,Recency
Customer ID,
12346,324 days 13:59:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [201]:
# Saatlerden kurtulma işlemi. Sadece günleri al dedik.
recency_df = temp_df["Recency"].apply(lambda x: x.days)

# Artık herbir müşterinin analiz yaptığımdan günden en son 
# satın alma gününe kadar geçen gün olarak süresi elimde.
recency_df.head()

Customer ID
12346    324
12347      1
12348     74
12349     17
12350    309
Name: Recency, dtype: int64

In [202]:
# Herbir müşterinin kaç faturası var.
# Bu müşterilerin toplam işlem sayısı önemli. 
# Bu dataframe üzerinden bir işlem daha gerçekleştirmek gerekir.

temp_df = df.groupby(["Customer ID", "Invoice"]).agg({"Invoice": "count"})
temp_df.head()


Invoice
Customer ID Invoice         
12346       541431         1
12347       537626        31
            542237        29
            549222        24
            556201        18

Frequency Hesaplama

In [203]:
# Herbir müşteri özelinde toplam yaptığı işlemler.
temp_df.groupby("Customer ID").agg({"Invoice": "sum"}).head()


,Invoice
Customer ID,
12346,1
12347,182
12348,31
12349,73
12350,17


In [204]:
freq_df = temp_df.groupby("Customer ID").agg({"Invoice":"sum"})
freq_df.rename(columns={"Invoice": "Frequency"}, inplace = True)
freq_df.head()


,Frequency
Customer ID,
12346,1
12347,182
12348,31
12349,73
12350,17


Monetary Hesaplama

In [205]:
monetary_df = df.groupby("Customer ID").agg({"TotalPrice": "sum"})
monetary_df.rename(columns = {"TotalPrice": "Monetary"}, inplace=1)
monetary_df.head()


,Monetary
Customer ID,
12346,77184
12347,4310
12348,1797
12349,1758
12350,334


In [206]:
print(recency_df.shape, freq_df.shape, monetary_df.shape)

(4339,) (4339, 1) (4339, 1)


In [207]:
rfm = pd.concat([recency_df, freq_df, monetary_df], axis=1)
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,324,1,77184
12347,1,182,4310
12348,74,31,1797
12349,17,73,1758
12350,309,17,334


RFM Değerlerini Skorlama

In [208]:
# qcut : Küçükten büyüğe sıralayıp, quartilerlara bölme işlemi
# yapar. Burada bizim için önemli olan küçük değerlerdir 
# bu yüzden onlara 5, büyük olan değerlere 1 atıyoruz

rfm["RecencyScore"] = pd.qcut(rfm["Recency"],5,labels = [5,4,3,2,1])
rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])
rfm.head()


,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
12346,324,1,77184,1,1,5
12347,1,182,4310,5,5,5
12348,74,31,1797,2,3,4
12349,17,73,1758,4,4,4
12350,309,17,334,1,2,2


In [209]:
# Skorları oluşturmak için astype ile değişkenleri stringe
# çevirmeliyiz ki bu değerleri yan yana koyduğumda anlamı olabilsin.
(rfm["RecencyScore"].astype(str) +
 rfm["FrequencyScore"].astype(str) +
 rfm["MonetaryScore"].astype(str)).head()


Customer ID
12346    115
12347    555
12348    234
12349    444
12350    122
dtype: object

In [210]:
rfm["RFM_SCORE"] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346,324,1,77184,1,1,5,115
12347,1,182,4310,5,5,5,555
12348,74,31,1797,2,3,4,234
12349,17,73,1758,4,4,4,444
12350,309,17,334,1,2,2,122


Müşteri Segmentleri

In [211]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

# Segment değişkenine Recency ve Frequency verdik. Bu iki 
# metrik özelinde tablomuzu oluşturuyoruz. Monetary, Frequency 
# ile aynı sayılabilir yakın değerler olduğu için.

rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()


,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,324,1,77184,1,1,5,115,Hibernating
12347,1,182,4310,5,5,5,555,Champions
12348,74,31,1797,2,3,4,234,At Risk
12349,17,73,1758,4,4,4,444,Loyal Customers
12350,309,17,334,1,2,2,122,Hibernating


In [212]:
# Grupları raporlayalım.
# Yukarıda müşterilere odaklanmıştık şimdi ise segmentlere.

rfm[["Segment","Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])


Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About to Sleep           51   316        16   316      449   316
At Risk                 165   583        56   583      986   583
Can't Loose             142    84       182    84     2371    84
Champions                 4   607       289   607     6950   607
Hibernating             208  1060        13  1060      537  1060
Loyal Customers          32   821       157   821     2835   821
Need Attention           51   210        41   210      846   210
New Customers             5    55         7    55     3742    55
Potential Loyalists      15   498        34   498      905   498
Promising                21   105         7   105      438   105

In [213]:
new_df = pd.DataFrame()
new_df["Need AttentionID"] = rfm[rfm["Segment"] == "Need Attention"].index
new_df.head()

,Need AttentionID
0,12372
1,12413
2,12446
3,12458
4,12475


In [ ]:
# Bir csv dosyasına kaydettik. Bu işlemden sonra 
# "need_attention" adında bir csv dosyayı oluşacaktır. 
# Bu ilgili departmanlara gönderilecek dosyadır.

# new_df.to_csv("need_attention.csv")